# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [30]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [31]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection

In [32]:
#your code here
spaceship = spaceship.dropna()
spaceship['Cabin'] = spaceship['Cabin'].str[0]
spaceship = spaceship.drop(columns = ['PassengerId', 'Name'])
spaceship_dummies = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP'], drop_first=True)
spaceship_dummies.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_True,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,True,False,False,True,False,False,False,False,False,False,False,True,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,False,False,False,False,False,False,False,True,False,False,False,True,False
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,True,False,False,False,False,False,False,False,False,False,False,True,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,True,False,False,False,False,False,False,False,False,False,False,True,False
4,16.0,303.0,70.0,151.0,565.0,2.0,True,False,False,False,False,False,False,False,True,False,False,False,True,False


**Perform Train Test Split**

In [33]:
#your code here
# Separate the features (X) and the target (y)
X = spaceship_dummies.drop(columns=['Transported'])  # Features (all columns except the target)
y = spaceship_dummies['Transported']  # Target (Transported column)

# Perform a train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the splits to confirm the sizes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (5284, 19)
X_test shape: (1322, 19)
y_train shape: (5284,)
y_test shape: (1322,)


**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [34]:
#your code here
# Bagging with replacement (default)
bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=100, max_samples=0.8, bootstrap=True, random_state=42
)
bagging_clf.fit(X_train, y_train)

# Predictions and accuracy
y_pred_bagging = bagging_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bagging)

print(f"Bagging Accuracy: {bagging_accuracy:.4f}")

Bagging Accuracy: 0.8071


In [35]:
# Pasting without replacement
pasting_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=100, max_samples=0.8, bootstrap=False, random_state=42
)
pasting_clf.fit(X_train, y_train)

# Predictions and accuracy
y_pred_pasting = pasting_clf.predict(X_test)
pasting_accuracy = accuracy_score(y_test, y_pred_pasting)

print(f"Pasting Accuracy: {pasting_accuracy:.4f}")

Pasting Accuracy: 0.8003


- Random Forests

In [36]:
#your code here
# Initialize the RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = rf_clf.predict(X_test)

# Evaluate the accuracy of the Random Forest model
rf_accuracy = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")

Random Forest Accuracy: 0.8048


- Gradient Boosting

In [37]:
#your code here
# Initialize the GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred_gb = gb_clf.predict(X_test)

# Evaluate the accuracy of the Gradient Boosting model
gb_accuracy = accuracy_score(y_test, y_pred_gb)

print(f"Gradient Boosting Accuracy: {gb_accuracy:.4f}")

Gradient Boosting Accuracy: 0.8071


- Adaptive Boosting

In [38]:
#your code here
# Initialize the AdaBoostClassifier with a default DecisionTreeClassifier as base
ada_clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42)

# Fit the model to the training data
ada_clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred_ada = ada_clf.predict(X_test)

# Evaluate the accuracy of the AdaBoost model
ada_accuracy = accuracy_score(y_test, y_pred_ada)

print(f"AdaBoost Accuracy: {ada_accuracy:.4f}")

c:\Users\alexa\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.7874


Which model is the best and why?

In [39]:
#comment here
print(f"Bagging Accuracy: {bagging_accuracy:.4f}")
print(f"Pasting Accuracy: {pasting_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"Gradient Boosting Accuracy: {gb_accuracy:.4f}")
print(f"AdaBoost Accuracy: {ada_accuracy:.4f}")

Bagging Accuracy: 0.8071
Pasting Accuracy: 0.8003
Random Forest Accuracy: 0.8048
Gradient Boosting Accuracy: 0.8071
AdaBoost Accuracy: 0.7874


I will choose Random Forest:
- Tend to reduce variance (less overfitting)
- Typically faster
- Provide useful insights into feature importance
- Robust, low-variance model that doesn’t overfit easily and can handle high-dimensional data well.